파이토치 튜토리얼

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # 물리적 순서대로 정렬
os.environ["CUDA_VISIBLE_DEVICES"] = "6"        # 나는 6번 GPU만 보이게 하겠다!

import torch
print(f"현재 사용 가능한 GPU 개수: {torch.cuda.device_count()}")
print(f"현재 선택된 GPU 번호: {torch.cuda.current_device()}")
print(f"장치 이름: {torch.cuda.get_device_name(0)}")

현재 사용 가능한 GPU 개수: 1
현재 선택된 GPU 번호: 0
장치 이름: NVIDIA GeForce RTX 3090


In [4]:
#### 매!!우!! 중요한 내용임!!! 이해하기

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5) # 1개의 흑백사진과, 6개의 필터. 그리고 그 필터 크기는 5*5크기
        self.conv2 = nn.Conv2d(6, 16, 5) # 6개의 입력 채널, 16개의 출력을 해주는 필터들. 그리고 그 필터 5*5크기
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension. 필요한 정보 조합해서 400개를 120개로 줄임. 학습된 가중치 기준.
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input)) # 입력을 6개의 필터에 통과시키고, 각각 레루 적용
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2)) # 2*2 크기의 필터, 근데 2칸씩 이동하고, 그 안에서 가장 큰 수만 남기는 필터. 사이즈는 가로세로 1/2씩
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2)) # s2를 16개의 필터 통과시키고, 렐류 적용.
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2) # c3를 2*2 필터로 1/2 크기로 줄임.
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1) # -->> 이걸 한 줄로 펴줌. 단, N의 정보는 빼고.
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4)) # 그 정보를 줄이고 렐루 적용. 근거는 데이터로부터 학습된 가중치 기준으로.
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5)) # 다시 줄이고 렐루 적용
        # Fully connected layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6) # 다시 줄이고 렐루 적용
        return output
# 굉장히 큰 수를 숫자 10개로 줄였다. 이제 10개의 이미지 종류중에서 뭐에 해당하는지 맞춰볼 수 있다.

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight. 필터 6개이고, 각 1개당 5*5이다. 

10
torch.Size([6, 1, 5, 5])


In [6]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out) 

tensor([[ 0.0246, -0.0499, -0.0019,  0.0676, -0.0008, -0.0411, -0.0044, -0.0597,
         -0.0438,  0.0834]], grad_fn=<AddmmBackward0>)


In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10)) # 확률적경사하강법 아님. 그냥 랜덤 수 넣어보면서 끝까지 전파되는건지 확인함.

In [8]:
output = net(input)
target = torch.randn(10)  # 정답 역할을 할 숫자 10개 랜덤으로 만들어봄. 연습용임. 실제 mnist라면 0,0,0,1,0,0,0,0,0,0 이런식으로 돼야함.
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.4767, grad_fn=<MseLossBackward0>)


input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

지금까지의 계산순서를 나타내었다.

In [9]:
# 각각 이전에 어떤 연산에 의해서 만들어졌는지를 알려줌. 실제로 우리가 원하는 연산을 해왔는지 체크해줌.
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [10]:
# 학습에서 쓰이는 grd는 그 step동안 backward로 만들어진 grad의 합이다. 그래서 계속 누적이 되는 것임.
net.zero_grad()     # zeroes the gradient buffers of all parameters
# 이걸 이해하면, 이후에 옵티마이저를 더 쉽게 이해할 수 있음.
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward() # 61,706차원의 가중치공간에서, 각각의 축으로 미분한 값을 모아 그라디언트 벡터를 61,706차원에 하나 만들었다.
#그리고 아미지 N장 중 미니배치 4장만 골라서 그라디언트를 구하면, 4개 함수의 그라디언트의 합으로 loss.backward가 만들어지고, 전파하면 됨.
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0034,  0.0054, -0.0072, -0.0007,  0.0010,  0.0137])


In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate) # 실제로 이렇게 돌아가지만, 다른 모델에서도 똑같이 돌아가게 하기 위해서 optimizer 사용.

In [12]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update